### Add model, connection and device to settings:

In [9]:
# библиотеки
import os
import sys
import inspect
import numpy as np
import re
import math
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from hybriddomain.envs.hs.model.model_main import ModelNet as Model
from hybriddomain.settings.settings_main import Settings
from ddeint import ddeint

# пути и названия файлов
modelFileName = 'problems/delay_equation_laba1'
conn_name = "connection.json"
device_conf_name = "devices.json"

#Ввод пароля для инициализации пользователя
model = Model()
model.io.loadFromFile(modelFileName)
settings = Settings(model, conn_name, device_conf_name, conn_name)

INFO:settings_main:device_conf_rpath
INFO:settings_main:devices.json


Please enter password for user imaslenikov:


 ·········


In [10]:
print("equations:")
print(model.equations)
print("params:")
print(model.params)
print("paramValues:")
print(model.paramValues)

equations:
[["U'= a*D[U,{x,2}] + c*(1-U(t-1)/b)*U"]]
params:
['a', 'b', 'c']
paramValues:
[{'a': 1, 'b': 1, 'c': 1.5}, {'a': 0.1, 'b': 1, 'c': 3}, {'a': 0.1, 'b': 10, 'c': 3}, {'a': 0.01, 'b': 10, 'c': 3}, {'a': 0.1, 'b': 10, 'c': 4}]


### Run solver:

In [11]:
# запуск процесса решения уравнения
from hybriddomain.solvers.hs.remoterun.progresses.progress_notebook import ProgressNotebook
from hybriddomain.solvers.hs.postproc.results.results_main import ResultPostprocNet as ResultPostproc
progress = ProgressNotebook(100, prefix='solving')

progress_params = ProgressNotebook(len(model.paramValues), prefix='param')
# display(progress_params.progress)

progress_solving = ProgressNotebook(100, prefix='solving')
# display(progress_solving.progress)
# progress_solving.succ(0)

result_postproc = ResultPostproc(modelFileName)

from hybriddomain.solvers.hs.remoterun.remoterun import remoteProjectRun

clear_output()
test_name = settings.paths['model']['name']
bTest = widgets.Button(description="run %s " % (test_name))

# parametrisation:
def run(event):
    model = Model()
    model.io.loadFromFile(modelFileName)
    
    for params_idx in range(len(model.paramValues)):
        
        progress_params.succ(params_idx)
        progress_solving.set_prefix("solving")
        
        # changing param:
        model.defaultParamsIndex = params_idx
        model.io.saveToFile(settings.paths["hd"]["json"])
        model = Model()
        model.io.loadFromFile(modelFileName)
        
        remoteProjectRun(settings, 1, progress_solving,remove_old=False)
        
        # result_postproc.extract_out(params_idx, results_params)
        result_postproc.rename_out(params_idx)
    display(Javascript("alert('Done');"))
    
bTest.on_click(run)

display(progress_params.progress)
display(progress_solving.progress)
 
display(bTest)

IntProgress(value=0, description='param: ', max=4)

IntProgress(value=0, description='solving: ', max=99)

Button(description='run delay_equation_laba1 ', style=ButtonStyle())

### Results:

In [12]:
model.readResults(result_format=1)
# available result files:
print("result files: ", model.results_paths)
# available plot files:
print("plot files: ", model.plots_paths)

result files:  {'u': ['problems/delay_equation_laba1/out/u_seq0.out', 'problems/delay_equation_laba1/out/u_seq1.out', 'problems/delay_equation_laba1/out/u_seq2.out', 'problems/delay_equation_laba1/out/u_seq3.out', 'problems/delay_equation_laba1/out/u_seq4.out']}
plot files:  {'U': ['problems/delay_equation_laba1/out/U_seq0.mp4', 'problems/delay_equation_laba1/out/U_seq1.mp4', 'problems/delay_equation_laba1/out/U_seq2.mp4', 'problems/delay_equation_laba1/out/U_seq3.mp4', 'problems/delay_equation_laba1/out/U_seq4.mp4']}


In [13]:
# объекте model хранится вся информация по решению задачи
# get results (for TRAC-11 result_format=1):
model.readResults(result_format=1)
model.results_paths
# available results:
print(model.results_arrays.keys())

dict_keys(['u'])


### Video:

In [14]:
# available plots:
model.plots_paths

{'U': ['problems/delay_equation_laba1/out/U_seq0.mp4',
  'problems/delay_equation_laba1/out/U_seq1.mp4',
  'problems/delay_equation_laba1/out/U_seq2.mp4',
  'problems/delay_equation_laba1/out/U_seq3.mp4',
  'problems/delay_equation_laba1/out/U_seq4.mp4']}

In [15]:
display(HTML(model.get_video('U')))
display(HTML(model.get_video('U',1)))
display(HTML(model.get_video('U',2)))
display(HTML(model.get_video('U',3)))
display(HTML(model.get_video('U',4)))

### U(x, t-fixed),V(x, t-fixed):

In [16]:
# отрисовывает результат
%matplotlib inline
time = model.results_arrays['u']['timevalues']

times = model.results_arrays['u']['timevalues']

result = model.results_arrays['u']['resvalues']
result_x = np.array([result[0][time][0][0] for time in times])
def f(t,x):
    result_x = np.array([result[x][time][0][t] for time in times])
    plt.plot(time,result_x)
w = interactive(f,t=(0,len(result[0][0][0])-1,1),x=(0,len(result)-1,1))
display(w)

plt.show()


interactive(children=(IntSlider(value=50, description='t'), IntSlider(value=2, description='x', max=4), Output…